In [ ]:
!pip install rasterio scikit-learn numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 37.6 MB/s eta 0:00:00


In [ ]:
!pip install pytesseract

## OCR

Pre processing to fine best parameters

In [ ]:
import cv2
import numpy as np
# img = cv2.imread('image.jpg')
# get grayscale image
def get_grayscale(img):
    cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return
# noise removal
def remove_noise(image):
    #cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 15)
    cv2.medianBlur(image,5)
    return
#thresholding
def thresholding(image):
    cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    return
#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    cv2.dilate(image, kernel, iterations = 1)
    return
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    cv2.erode(image, kernel, iterations = 1)
    return

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)
    return

#canny edge detection
def canny(image):
    cv2.Canny(image, 100, 200)
    return

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

#template matching
def match_template(image, template):
    cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
    return

In [ ]:
# We used  the original image, we need change just the name of the main image,
#img = cv2.imread("main_2.tiff")  # shape (2516, 4003, 3)
img = cv2.imread("main_1.tif")   #shape (2524, 4033, 3)
# cv2.imshow("img" , img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
print(img.shape)
# print(img2.shape)

(2524, 4033, 3)


In [ ]:
inverted_imge = cv2.bitwise_not(img)
cv2.imwrite("inverted_main_32.tiff" , inverted_imge)
cv2.imwrite("inverted_main_32.png" , inverted_imge)

# cv2.imwrite("inverted_main_39.tiff" , inverted_imge)
# cv2.imwrite("inverted_main_39.png" , inverted_imge)

# cv2.imshow("inverted_imge" , inverted_imge)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True

In [ ]:
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imwrite("gray_img_main_32.png" , gray_img)
# cv2.imwrite("gray_img_main_39.png" , gray_img)

# cv2.imshow("gray_img" , gray_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

True

In [ ]:
#we try to find the best parameter for thresholding

import cv2
import os

thresholds = [50, 60, 70, 80, 90, 100, 120, 130]
upper_limits = [100, 150, 160, 180, 190, 200, 230]

for lower, upper in zip(thresholds, upper_limits):
    thresh, im_bw = cv2.threshold(gray_img, lower, upper, cv2.THRESH_BINARY)

    filename = f"threshold_{lower}_{upper}_32.png"
#     filename = f"threshold_{lower}_{upper}_39.png"

    cv2.imwrite(filename, im_bw)
    print(im_bw.shape)

(2524, 4033)
(2524, 4033)
(2524, 4033)
(2524, 4033)
(2524, 4033)
(2524, 4033)
(2524, 4033)


In [ ]:
#remove noise
def noise_remover(image):
    import numpy as np
    kernal = np.ones((2,2) ,np.uint8)
    image = cv2.dilate(image, kernal , iterations =1)
    kernal = np.ones((2,2) , np.uint8)
    image = cv2.erode(image , kernal, iterations = 1)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE  , kernal )
#     image = cv2.medianBlur(image , 3)
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.GaussianBlur(image, (5, 5), 0)
    return image

no_noise = noise_remover (im_bw)
cv2.imwrite("no_noise2_32.png" , no_noise)
# cv2.imwrite("no_noise2_39.png" , no_noise)

True

In [ ]:
# Install pytesseract and Tesseract OCR
!sudo apt-get install tesseract-ocr
!pip install pytesseract

In [ ]:
import cv2
import numpy as np
import pytesseract
from PIL import Image
from pytesseract import Output

# noise remove fun
def noise_remover(image):
    kernal = np.ones((2, 2), np.uint8)
    image = cv2.dilate(image, kernal, iterations=1)
    kernal = np.ones((2, 2), np.uint8)
    image = cv2.erode(image, kernal, iterations=1)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernal)
    image = cv2.GaussianBlur(image, (5, 5), 0)
    return image

thresholds = [50, 60, 70, 80, 90, 100, 120, 130]
upper_limits = [100, 150, 160, 180, 190, 200, 230]

for lower, upper in zip(thresholds, upper_limits):
    thresh, im_bw = cv2.threshold(gray_img, lower, upper, cv2.THRESH_BINARY)

#apply noise removal to the threshold image
    no_noise = noise_remover(im_bw)


    filename = f"threshold_{lower}_{upper}_no_noise_32.png"
#     filename = f"threshold_{lower}_{upper}_no_noise_39.png"
    cv2.imwrite(filename, no_noise)

# OCR on the thresholded image with no noise
    myconfig = r"--psm 11 --oem 3"
    data = pytesseract.image_to_data(Image.fromarray(no_noise), config=myconfig, output_type=Output.DICT)
#     print(data['text'])
    word_list = [word for word in data['text'] if word.strip() != '']
#     print(word_list )

    # Print the results
    print(f"Threshold: {lower} - {upper}")
    for i in range(len(data['text'])):
        text = data['text'][i]
        left = data['left'][i]
        top = data['top'][i]
        width = data['width'][i]
        height = data['height'][i]
        print(f"Text: {text}, Bounding Box: (left: {left}, top: {top}, width: {width}, height: {height})")

    filename = f"threshold_{lower}_{upper}_no_noise_32.png"
#     filename = f"threshold_{lower}_{upper}_no_noise_39.png"
    cv2.imwrite(filename, no_noise)

Streaming output truncated to the last 5000 lines.
Text: , Bounding Box: (left: 244, top: 1215, width: 39, height: 15)
Text: “hd, Bounding Box: (left: 244, top: 1215, width: 39, height: 15)
Text: , Bounding Box: (left: 365, top: 1240, width: 20, height: 5)
Text: , Bounding Box: (left: 365, top: 1240, width: 20, height: 5)
Text: , Bounding Box: (left: 365, top: 1240, width: 20, height: 5)
Text: —, Bounding Box: (left: 365, top: 1240, width: 20, height: 5)
Text: , Bounding Box: (left: 2707, top: 1228, width: 61, height: 9)
Text: , Bounding Box: (left: 2707, top: 1228, width: 61, height: 9)
Text: , Bounding Box: (left: 2707, top: 1228, width: 61, height: 9)
Text: a, Bounding Box: (left: 2707, top: 1228, width: 61, height: 9)
Text: , Bounding Box: (left: 3367, top: 1210, width: 44, height: 19)
Text: , Bounding Box: (left: 3367, top: 1210, width: 44, height: 19)
Text: , Bounding Box: (left: 3367, top: 1210, width: 44, height: 19)
Text: is, Bounding Box: (left: 3367, top: 1199, width: 16, he

In [ ]:
# ITS HOWS THE ALL TEXT in all images (MORE That 3 Char for each words)

import cv2
import numpy as np
from pytesseract import pytesseract
from PIL import Image
from pytesseract import Output
import pandas as pd
from google.colab.patches import cv2_imshow

# noise removal function
def noise_remover(image):
    kernel = np.ones((2, 2), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    image = cv2.GaussianBlur(image, (5, 5), 0)
    return image

thresholds = [50, 60, 70, 80, 90, 100, 120, 130]
upper_limits = [100, 150, 160, 180, 190, 200, 230]

#image display size
display_width, display_height = 4033 ,2524   # for 32    (2524, 4033)
# display_width, display_height = 4003 ,2516   # for 39


for lower, upper in zip(thresholds, upper_limits):
    thresh, im_bw = cv2.threshold(gray_img, lower, upper, cv2.THRESH_BINARY)
    no_noise = noise_remover(im_bw)
    display_img = cv2.resize(no_noise, (display_width, display_height))

    data = pytesseract.image_to_data(Image.fromarray(no_noise), config=r"--psm 11 --oem 3", output_type=Output.DICT)

    # DataFrame to store
    df = pd.DataFrame(data)

    for i in range(len(data['text'])):
        text = data['text'][i]
        if len(text) >= 3:  # Exclude words with less than 3 chars
            left = int(data['left'][i] * display_width / no_noise.shape[1])
            top = int(data['top'][i] * display_height / no_noise.shape[0])
            width = int(data['width'][i] * display_width / no_noise.shape[1])
            height = int(data['height'][i] * display_height / no_noise.shape[0])

            cv2.rectangle(display_img, (left, top), (left + width, top + height), (0, 255, 0), 2)
            cv2.putText(display_img, text, (left, top + height + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
#
    window_title = f"Threshold: {lower} - Upper Limit: {upper}_32"
#     window_title = f"Threshold: {lower} - Upper Limit: {upper}_39"
    # cv2.imshow(window_title, display_img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    filename = f"threshold_{lower}_{upper}_with_boxes_and_text_32.png"
#     filename = f"threshold_{lower}_{upper}_with_boxes_and_text_39.png"
    cv2.imwrite(filename, display_img)

    #DataFrame to CSV
    csv_filename = f"threshold_{lower}_{upper}_data_32.csv"
#     csv_filename = f"threshold_{lower}_{upper}_data_39.csv"
    df.to_csv(csv_filename, index=False)

print("FInished")

FInished
